## Tables to create

Key tables
* alignment
* death
* roles
* settings

Data tables
* player (just player name & id)
* game (general stats about each game)
* gms (gms for each game)
* playergame (each player's stats in each game)
* player_roles (roles for a row in playergame)

In [1]:
import pandas as pd 
import numpy as np

In [2]:
# KEY TABLES

alignment = pd.read_csv("data/keys/alignment.csv").reset_index().rename(columns={"index": "alignment_id"})

print(alignment.head())

setting = pd.read_csv("data/keys/setting.csv")

print(setting.head())

death = pd.read_csv("data/keys/death.csv").reset_index().rename(columns={"index": "death_id"})

death.head()

   alignment_id alignment_char                     alignment_desc  is_elim  \
0             0              G                               Good    False   
1             1              E  Evil (Prim) (team kill / no conv)     True   
2             2              S             Secondary Evil Faction     True   
3             3              T              Tertiary Evil Faction     True   
4             4              ?            Evil Converter (w/kill)     True   

   is_evil  has_kill  has_convert  was_converted  
0    False     False        False          False  
1     True      True        False          False  
2     True      True        False          False  
3     True      True        False          False  
4     True      True         True          False  
         setting  is_sanderson  is_cosmere
0  Cross-Cosmere          True        True
1            Sel          True        True
2       Scadrial          True        True
3        Nalthis          True        True
4         

,death_id,death_char,death_desc,dying_alignment,killer_alignment,non-kills
0,0,S,Survived,none,none,NaN
1,1,E,Killed by Eliminators,not E,E,NaN
2,2,X,Exed,any,none,NaN
3,3,V,Elim killed by Village,E,V,NaN
4,4,F,Villager Killed by Village (Friendly Fire),V,V,NaN


In [3]:
# IMPORTING DATA TAB
data = pd.read_csv("data/original_data/orig_data_040622.csv")

data.rename(inplace=True, columns={
    "ID": "pg_id",
    "Player": "player_name",
    "Alignment": "alignment_char",
    "Faction Outcome": "win", 
    "First Hit": "first_hit", 
    "Last Hit": "last_hit",
    "# of Hits": "num_hits",
    "Death/\n Survival": "death_char", # ???
    "Inactive": "inactive",
    "Mod": "IM",
    "Game Type": "game_format",
    "Game #": "game_num",
    "Unnamed: 15": "game_string", # these depend on other cols. name in sheet pls
    "Role": "role",
    "Unnamed: 18": "secondary_role",
    "Main?": "main_gm"
    })

data = data.filter([
    "pg_id", 
    "player_name",
    "alignment_char",
    "win",
    "first_hit", "last_hit", "num_hits",
    "death_char",
    "inactive",
    "GM", "main_gm", "IM", # "Spec"
    "game_string", #"game_format", "game_num",
    "role", "secondary_role"
    ])

data.head()

# more cleaning here? 

,pg_id,player_name,alignment_char,win,first_hit,last_hit,num_hits,death_char,inactive,GM,main_gm,IM,game_string,role,secondary_role
0,1,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,LG1,GM,Roles that are unique and don't fit
1,2,Cracknut,G,L,1.0,1.0,1.0,E,NaN,NaN,NaN,NaN,LG1,Tineye,MR1/LG8/MR7 need work
2,3,Edgedancer,G,L,2.0,2.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Cop,NaN
3,4,Mailliw73,G,L,4.0,4.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Regular,NaN
4,5,Aether,E,W,4.0,4.0,1.0,V,NaN,NaN,NaN,NaN,LG1,Smoker,NaN


In [53]:
# IMPORTING GAME TAB

# header = 1 because there's an extra header row on top w categories
game = pd.read_csv("data/original_data/orig_game_040622.csv", header=1)

# cols wanted: id, format, number, string, anon num,
# mech balance, dist balance, IM id, start date, end date
# num cycles, num posts, setting id
# complexity, fundamentals, role madness

# cols still needed: format, number, mech balance, dist balance, 
# IM id, start date, end date, setting id
# IM id probably pulled from pg

game.reset_index(inplace=True)

game.columns

Index(['index', 'format', 'num', 'auto?', '# Players', '# Cycles', '# Posts',
       'Fundamentals', 'Winner', 'Complexity', 'Role Madness', 'Anon', 'Mech',
       'Dist', 'Broken', '# Elims', 'Elim %', '# Xed', '# Vig'd', '# Deaths',
       '# Survivors', '# E killed', '# M killed', '# N killed', '# V killed',
       '# FF', '# Xed.1', '# I killed', '# pinchhitters', '# Spectators',
       'GM(s)', 'IM', 'Title/Link', 'Sanderson? ', 'World', 'start_date',
       'end_date', 'link'],
      dtype='object')

In [54]:

# ! Need to be careful about sheet renaming cols - some kind of check?
game.rename(inplace=True, columns={
    "index": "game_id",
    "format": "game_format",
    "num": "game_num", 
    "auto?": "game_string", # optional? 
    #"# Players": "num_players", # optional
    "# Cycles": "num_cycles",
    "# Posts": "num_posts",
    "Fundamentals": "fundamentals", 
    "Role Madness": "role_madness",
    #"Winner": "winner", # optional
    "Complexity": "complexity",
    #"Broken": "broken", # TODO make into mech/dist once that's set in sheet
    "Mech": "mechanics_balance", # blank rn!!
    "Dist": "distribution_balance",
    "World": "setting", # need to make setting table and replace this w setting_id
    "Anon": "anon_num", 
    "Title/Link": "title", # need to get link separately. hidden col? 
    # LINK
    "start_date": "start_date",
    "end_date": "end_date"
    })

print(game.columns)

# remaining columns are just calculated from data test
game = game.filter([
    "game_id",
    "game_format", "game_num", "game_string",
    # "num_players",
    #"IM", # currently blank - pulling from Data instead
    "num_cycles", 
    "num_posts", 
    "fundamentals", "role_madness", "complexity", 
    #"winner", 
    "mechanics_balance", "distribution_balance",
    #"broken", 
    "setting",
    "anon_num",
    "title",
    "start_date", "end_date"
])


game.tail()
# check for almost all NaN rows! extra checkboxes cause this

Index(['game_id', 'game_format', 'game_num', 'game_string', '# Players',
       'num_cycles', 'num_posts', 'fundamentals', 'Winner', 'complexity',
       'role_madness', 'anon_num', 'mechanics_balance', 'distribution_balance',
       'Broken', '# Elims', 'Elim %', '# Xed', '# Vig'd', '# Deaths',
       '# Survivors', '# E killed', '# M killed', '# N killed', '# V killed',
       '# FF', '# Xed.1', '# I killed', '# pinchhitters', '# Spectators',
       'GM(s)', 'IM', 'title', 'Sanderson? ', 'setting', 'start_date',
       'end_date', 'link'],
      dtype='object')


,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,anon_num,title,start_date,end_date
208,208,LG,83.0,LG83,6,NaN,V/E,False,NaN,NaN,NaN,NaN,NaN,The Survivor,NaN,NaN
209,209,BT,1.0,BT1,4,NaN,V/E,False,NaN,NaN,NaN,Cross-Cosmere,NaN,Auction of Stories,NaN,NaN
210,210,QF,59.0,QF59,7,NaN,V/E,False,NaN,NaN,NaN,Roshar,NaN,Bachelor - Roshar Edition!,NaN,NaN
211,211,MR,57.0,MR57,NaN,NaN,V/E,False,Semi Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,212,LG,84.0,LG84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


END CLEANING 

# SQL table making

In [6]:
# PLAYER - DONE
unique_players = data["player_name"].unique()

player = pd.DataFrame({"player_id": range(len(unique_players)), "player_name": unique_players})

player.head()


,player_id,player_name
0,0,Meta
1,1,Cracknut
2,2,Edgedancer
3,3,Mailliw73
4,4,Aether


In [7]:
# SETTINGS - DONE

unique_worlds = game["setting"].unique()

new_settings = []

for world in unique_worlds:
    if world not in setting["setting"].unique():
        if (str(world) == "nan"): # ??
            print("what") 
            continue # check
        new_settings.append({"setting": world, "is_sanderson": False, "is_cosmere": False})
        print(f"inserted {world}")

new_settings_df = pd.DataFrame(new_settings)

setting = pd.concat([setting, new_settings_df], ignore_index=True).reset_index().rename(columns={"index": "setting_id"})

setting.tail()

# TODO: check this against key table settings for worlds not in it
# also, don't have it insert NaN
# for each loc not there, put down as non-Sanderson and print that you're doing so

# do the same check with unique death and alignment tables, but don't insert by default - just print thing

inserted Hunger Games
inserted nan
inserted Game of Thrones
inserted KKC
inserted Death Note
inserted Codex Alera
inserted Pokemon
inserted Black Prism
inserted Red Rising
inserted Discworld
inserted Elder Scrolls
inserted My Little Pony
inserted Les Miserables
inserted Gunnerkrigg Court
inserted Lord of the Rings
inserted Kingkiller Chronicles
inserted Princess Bride
inserted Zelda
inserted Firefly


,setting_id,setting,is_sanderson,is_cosmere
30,30,Lord of the Rings,False,False
31,31,Kingkiller Chronicles,False,False
32,32,Princess Bride,False,False
33,33,Zelda,False,False
34,34,Firefly,False,False


In [31]:
# ALIGNMENT / DEATH - NOPE

al = data["alignment_char"].unique()

for a in al:
    if a not in alignment["alignment_char"].unique():
        print(a)



de = data["death_char"].unique()

for d in de:
    if d not in death["death_char"].unique():
        print(d)


# TODO fix rows with these things (or could do in SQL insertion or merge)

nan
nan
L?
?
V/M
L
V/E
X/E


In [21]:
data.columns

Index(['pg_id', 'player_name', 'alignment_char', 'win', 'first_hit',
       'last_hit', 'num_hits', 'death_char', 'inactive', 'GM', 'main_gm', 'IM',
       'game_string', 'role', 'secondary_role', 'player_id'],
      dtype='object')

In [26]:
death

,death_id,death_char,death_desc,dying_alignment,killer_alignment,non-kills
0,0,S,Survived,none,none,NaN
1,1,E,Killed by Eliminators,not E,E,NaN
2,2,X,Exed,any,none,NaN
3,3,V,Elim killed by Village,E,V,NaN
4,4,F,Villager Killed by Village (Friendly Fire),V,V,NaN
5,5,M,Killed by Neutral Evil,any,M,NaN
6,6,N,Neutral Killed,any,N,NaN
7,7,I,Inactive Killed,any,none,Y
8,8,O,Out of Commission (alive but not counting),any,any,Y
9,9,D,Dropped Out (good reason),any,none,Y


In [29]:
# TODO id replacements:
# in PG - player_id, game_id, alignment_id, death_id

s = pd.Series(player.set_index("player_name")["player_id"])
data["player_id"] = data["player_name"].map(s)

s = pd.Series(game.set_index("game_string")["game_id"])
data["game_id"] = data["game_string"].map(s)

s = pd.Series(alignment.set_index("alignment_char")["alignment_id"])
data["alignment_id"] = data["alignment_char"].map(s)

s = pd.Series(death.set_index("death_char")["death_id"])
data["death_id"] = data["death_char"].map(s)

data.head()


,pg_id,player_name,alignment_char,win,first_hit,last_hit,num_hits,death_char,inactive,GM,main_gm,IM,game_string,role,secondary_role,player_id,game_id,alignment_id,death_id
0,1,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,NaN,LG1,GM,Roles that are unique and don't fit,0,0.0,NaN,NaN
1,2,Cracknut,G,L,1.0,1.0,1.0,E,NaN,NaN,NaN,NaN,LG1,Tineye,MR1/LG8/MR7 need work,1,0.0,0.0,1.0
2,3,Edgedancer,G,L,2.0,2.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Cop,NaN,2,0.0,0.0,2.0
3,4,Mailliw73,G,L,4.0,4.0,1.0,X,NaN,NaN,NaN,NaN,LG1,Regular,NaN,3,0.0,0.0,2.0
4,5,Aether,E,W,4.0,4.0,1.0,V,NaN,NaN,NaN,NaN,LG1,Smoker,NaN,4,0.0,1.0,3.0


In [55]:
s = pd.Series(setting.set_index("setting")["setting_id"])
game["setting_id"] = game["setting"].map(s)

game.head()

,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,anon_num,title,start_date,end_date,setting_id
0,0,LG,1.0,LG1,4,462.0,V/E,False,Standard,NaN,NaN,Scadrial,NaN,In the Wake of Koloss,NaN,NaN,2
1,1,LG,2.0,LG2,11,1232.0,CON,False,Standard,NaN,NaN,Scadrial,NaN,Devil's Den,NaN,NaN,2
2,2,LG,3.0,LG3,7,739.0,V/E,False,Standard,NaN,NaN,Scadrial,NaN,Blackwater Village,NaN,NaN,2
3,3,LG,4.0,LG4,9,1029.0,V/E,False,Semi Standard,NaN,NaN,Nalthis,NaN,Colours of War,NaN,NaN,3
4,4,QF,1.0,QF1,4,118.0,V/E,False,Basic,NaN,NaN,Sel,NaN,Jeskeri Mysteries,NaN,NaN,1


In [56]:
# Pull rest of stuff into game table from PG: IM id

IM_list = data[data["IM"] == "Y"].filter(["player_id", "game_id"]).convert_dtypes()

game = game.merge(IM_list, on="game_id", how="left").rename(columns={"player_id": "IM_id"})

game.head()
# export games to csv (optionally)

,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,anon_num,title,start_date,end_date,setting_id,IM_id
0,0,LG,1.0,LG1,4,462.0,V/E,False,Standard,NaN,NaN,Scadrial,NaN,In the Wake of Koloss,NaN,NaN,2,<NA>
1,1,LG,2.0,LG2,11,1232.0,CON,False,Standard,NaN,NaN,Scadrial,NaN,Devil's Den,NaN,NaN,2,<NA>
2,2,LG,3.0,LG3,7,739.0,V/E,False,Standard,NaN,NaN,Scadrial,NaN,Blackwater Village,NaN,NaN,2,<NA>
3,3,LG,4.0,LG4,9,1029.0,V/E,False,Semi Standard,NaN,NaN,Nalthis,NaN,Colours of War,NaN,NaN,3,<NA>
4,4,QF,1.0,QF1,4,118.0,V/E,False,Basic,NaN,NaN,Sel,NaN,Jeskeri Mysteries,NaN,NaN,1,<NA>


In [36]:
# make GMs table from PG

gms = data[data["GM"] == "Y"].filter(["player_id", "game_id", "main_gm"])

#gms.rename(columns={"player_id": "gm_id"}, inplace=True)

# ! MAIN GM

gms["main_gm"] = gms["main_gm"].map({"Y": True, "nan": False, np.nan: False})

gms.head()

# optionally export gms table

,player_id,game_id,main_gm
0,0,0.0,True
20,0,1.0,True
62,15,2.0,True
83,5,3.0,True
135,48,4.0,True


In [14]:
# if roles are in, make roles table here

In [38]:
data.columns

Index(['pg_id', 'player_name', 'alignment_char', 'win', 'first_hit',
       'last_hit', 'num_hits', 'death_char', 'inactive', 'GM', 'main_gm', 'IM',
       'game_string', 'role', 'secondary_role', 'player_id', 'game_id',
       'alignment_id', 'death_id'],
      dtype='object')

In [65]:
# clean PG - take out GM, spec, IM, ??


playergame = data[data["GM"] != "Y"]
playergame = playergame[playergame["IM"] != "Y"]
# playergame = playergame[playergame["Spec"] != "Y"]


playergame["win"] = playergame["win"].map({"W": True, "L": False, "D": "NULL"})
playergame["inactive"] = playergame["inactive"].map({"Y": True, "NaN": False, np.nan: False})

playergame = playergame.filter(['pg_id', 'win', 'first_hit', 'last_hit', 'num_hits', 'inactive', 'player_id', 'game_id', 'alignment_id', 'death_id'])

playergame = playergame.convert_dtypes()

playergame.head()

# optionally export playergame table

,pg_id,win,first_hit,last_hit,num_hits,inactive,player_id,game_id,alignment_id,death_id
1,2,False,1,1,1,False,1,0,0,1
2,3,False,2,2,1,False,2,0,0,2
3,4,False,4,4,1,False,3,0,0,2
4,5,True,4,4,1,False,4,0,1,3
5,6,True,<NA>,<NA>,0,False,5,0,1,0


In [57]:
game.columns

Index(['game_id', 'game_format', 'game_num', 'game_string', 'num_cycles',
       'num_posts', 'fundamentals', 'role_madness', 'complexity',
       'mechanics_balance', 'distribution_balance', 'setting', 'anon_num',
       'title', 'start_date', 'end_date', 'setting_id', 'IM_id'],
      dtype='object')

In [62]:
game = game.convert_dtypes()
game.head()

,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,anon_num,title,start_date,end_date,setting_id,IM_id
0,0,LG,1.0,LG1,4,462,V/E,False,Standard,<NA>,<NA>,Scadrial,<NA>,In the Wake of Koloss,<NA>,<NA>,2,<NA>
1,1,LG,2.0,LG2,11,1232,CON,False,Standard,<NA>,<NA>,Scadrial,<NA>,Devil's Den,<NA>,<NA>,2,<NA>
2,2,LG,3.0,LG3,7,739,V/E,False,Standard,<NA>,<NA>,Scadrial,<NA>,Blackwater Village,<NA>,<NA>,2,<NA>
3,3,LG,4.0,LG4,9,1029,V/E,False,Semi Standard,<NA>,<NA>,Nalthis,<NA>,Colours of War,<NA>,<NA>,3,<NA>
4,4,QF,1.0,QF1,4,118,V/E,False,Basic,<NA>,<NA>,Sel,<NA>,Jeskeri Mysteries,<NA>,<NA>,1,<NA>


In [ ]:
# OPTIONAL: CSV CREATION 
player.to_csv("data/player.csv", index=False)
# playergame, game, alignment, death, blah blah blah

In [66]:
playergame.dtypes

pg_id             Int64
win              object
first_hit         Int64
last_hit          Int64
num_hits          Int64
inactive        boolean
player_id         Int64
game_id           Int64
alignment_id      Int64
death_id          Int64
dtype: object

In [45]:
game.head() 
# can remove is_sanderson and is_cosmere

,game_id,game_format,game_num,game_string,num_cycles,num_posts,fundamentals,role_madness,complexity,mechanics_balance,distribution_balance,setting,title,start_date,end_date,setting_id
0,0,LG,1.0,LG1,4,462.0,V/E,False,Standard,NaN,NaN,Scadrial,In the Wake of Koloss,NaN,NaN,2
1,1,LG,2.0,LG2,11,1232.0,CON,False,Standard,NaN,NaN,Scadrial,Devil's Den,NaN,NaN,2
2,2,LG,3.0,LG3,7,739.0,V/E,False,Standard,NaN,NaN,Scadrial,Blackwater Village,NaN,NaN,2
3,3,LG,4.0,LG4,9,1029.0,V/E,False,Semi Standard,NaN,NaN,Nalthis,Colours of War,NaN,NaN,3
4,4,QF,1.0,QF1,4,118.0,V/E,False,Basic,NaN,NaN,Sel,Jeskeri Mysteries,NaN,NaN,1


In [58]:
alignment.head()

,alignment_id,alignment_char,alignment_desc,is_elim,is_evil,has_kill,has_convert,was_converted
0,0,G,Good,False,False,False,False,False
1,1,E,Evil (Prim) (team kill / no conv),True,True,True,False,False
2,2,S,Secondary Evil Faction,True,True,True,False,False
3,3,T,Tertiary Evil Faction,True,True,True,False,False
4,4,?,Evil Converter (w/kill),True,True,True,True,False


In [59]:
death.head()

,death_id,death_char,death_desc,dying_alignment,killer_alignment,non-kills
0,0,S,Survived,none,none,NaN
1,1,E,Killed by Eliminators,not E,E,NaN
2,2,X,Exed,any,none,NaN
3,3,V,Elim killed by Village,E,V,NaN
4,4,F,Villager Killed by Village (Friendly Fire),V,V,NaN


In [60]:
print(player.head())
print(gms.head())
print(setting.head())

   player_id player_name
0          0        Meta
1          1    Cracknut
2          2  Edgedancer
3          3   Mailliw73
4          4      Aether
     player_id  game_id  main_gm
0            0      0.0     True
20           0      1.0     True
62          15      2.0     True
83           5      3.0     True
135         48      4.0     True
   setting_id        setting  is_sanderson  is_cosmere
0           0  Cross-Cosmere          True        True
1           1            Sel          True        True
2           2       Scadrial          True        True
3           3        Nalthis          True        True
4           4         Roshar          True        True


In [72]:
# removing bad characters

player["player_name"] = player["player_name"].str.replace("'", "") 

game["title"] = game["title"].str.replace("'", "")

player.iloc[290:310]


,player_id,player_name
290,290,Feruchemical Skybreaker
291,291,Zillah
292,292,Experience
293,293,Ironfire
294,294,Lord_Silberfarben
295,295,Karnage
296,296,The_Archivist
297,297,Mist
298,298,Shard of Reading
299,299,Matrims_Dice


## Tracking SQL appropriateness: 
* player: DONE
* setting: DONE  (remove nan row)
* alignment: maybe? 
* death: DONE (have extra cols not being inserted)
* **game**: nooo (missing some cols; bad types - see below)
* gms: DONE ish (not currently marking main GM bool, but SQL has a default and idc)
* **playergame**: nope (below)

### Problems with game 
* Missing columns: 
    - Mech and dist balance (not yet in sheet; just make all null? )
    - start and end date (not yet in sheet)
    - link (not in sheet in a helpful way)
* Type problems: 
    - num_posts is a float (??)

### Problems with playergame 
* Type problems: 
    - first, last, and num hits are all floats and should be ints
    - ids for game, alignment, and death should be ints
* pinchhitter doesn't exist

### Where to check for bad characters:
* game.title 
* player.player_name
* game.fundamentals?

### Other
* PG needs to be cleaned to remove things not in death / alignment tables; want to be able to do constraints also on complexity / etc 

At this point should have the tables:
* playergame
* games (game)
* player
* gms
* alignment
* death
* settings

time for...
## SQL Insertion


In [20]:
import psycopg2
db = 1

ModuleNotFoundError: No module named 'psycopg2'

In [30]:
if db != 0:
    cnx = psycopg2.connect(user="isanford_123", database="isanford", host="localhost", password="12345678")
    cursor = cnx.cursor()


OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "isanford"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "isanford"


In [28]:
# PLAYER
#db = 0
cc = 0 

for row in player.iterrows():
    ins = "insert into player (player_id, player_name) values ({}, '{}');"
    ins = ins.format(row[1]["player_id"], row[1]["player_name"])

    if db == 1:
        #try: 
            cursor.execute(ins)
        # except:
        #     print(row)
        #     print(ins)
        #     cc = cc + 1

    else: 
        print(ins)
    
    break

print(f"Num errors: {cc}")

InsufficientPrivilege: permission denied for table player


In [24]:
alignment.head()

,alignment_id,alignment_char,alignment_desc,is_elim,is_evil,has_kill,has_convert,was_converted
0,0,G,Good,False,False,False,False,False
1,1,E,Evil (Prim) (team kill / no conv),True,True,True,False,False
2,2,S,Secondary Evil Faction,True,True,True,False,False
3,3,T,Tertiary Evil Faction,True,True,True,False,False
4,4,?,Evil Converter (w/kill),True,True,True,True,False


In [25]:
# KEY TABLES

# ALIGNMENT

cc = 0 

for row in alignment.iterrows():
    tup = row[1]
    ins = "INSERT INTO alignment (alignment_id, alignment_char, alignment_desc, is_elim, is_evil, has_kill, has_convert) VALUES('{}','{}','{}','{}','{}','{}','{}', '{}')"

    ins = ins.format(tup["alignment_id"], tup["alignment_char"], tup["alignment_desc"], tup["is_elim"], tup["is_evil"], tup["has_kill"], tup["has_convert"], tup["was_converted"])

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    break

print(f"Num errors: {cc}")

INSERT INTO alignment (alignment_id, alignment_char, alignment_desc, is_elim, is_evil, has_kill, has_convert) VALUES('0','G','Good','False','False','False','False', 'False')
Num errors: 0


In [26]:
# KEY TABLES

cc = 0 

# optional: also add other cols if they're reasonable

for row in death.iterrows():
    tup = row[1]
    ins = "INSERT INTO death (death_id, death_char, death_desc) VALUES('{}','{}','{}')"

    ins = ins.format(tup["death_id"], tup["death_char"], tup["death_desc"])

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    break

print(f"Num errors: {cc}")

INSERT INTO death (death_id, death_char, death_desc) VALUES('0','S','Survived')
Num errors: 0


In [27]:
settings.tail()

,setting_id,world,is_sanderson,is_cosmere
30,30,Lord of the Rings,False,False
31,31,Kingkiller Chronicles,False,False
32,32,Princess Bride,False,False
33,33,Zelda,False,False
34,34,Firefly,False,False


In [28]:
# KEY TABLES

cc = 0 

# optional: also add other cols if they're reasonable

for row in settings.iterrows():
    tup = row[1]
    ins = "INSERT INTO settings (setting_id, world, is_sanderson, is_cosmere) VALUES('{}','{}','{}','{}')"

    ins = ins.format(tup["setting_id"], tup["world"], tup["is_sanderson"], tup["is_cosmere"])

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
        
    break

print(f"Num errors: {cc}")

INSERT INTO settings (setting_id, world, is_sanderson, is_cosmere) VALUES('0','Cross-Cosmere','True','True')
Num errors: 0


In [29]:
gms.head()

,player_id,game_id
0,0,0.0
20,0,1.0
62,15,2.0
83,5,3.0
135,48,4.0


In [30]:
# KEY TABLES

cc = 0 

# optional: also add other cols if they're reasonable

for row in gms.iterrows():
    tup = row[1]
    ins = "INSERT INTO gms (game_id, player_id) VALUES('{}','{}')"

    ins = ins.format(tup["game_id"], tup["player_id"])

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    break

print(f"Num errors: {cc}")

INSERT INTO gms (game_id, player_id) VALUES('0.0','0.0')
Num errors: 0


In [31]:
# KEY TABLES

cc = 0 

for row in game.iterrows():
    tup = row[1]
    ins = "INSERT INTO game (game_id, game_format, game_number, game_string, anon_num, mechanics_balance, distribution_balance, IM_id, start_date, end_date, num_cycles, num_posts, setting_id, complexity, fundamentals, role_madness) VALUES('{}','{}','{}','{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}')"

    ins = ins.format(
        tup["game_id"], 
        tup["game_format"], 
        tup["game_number"], 
        tup["game_string"], 
        tup["anon_num"],
        tup["mechanics_balance"], #not yet
        tup["distribution_balance"], # not yet
        tup["IM_id"],
        tup["start_date"], # not yet
        tup["end_date"], # not yet
        tup["num_cycles"], 
        tup["num_posts"],
        tup["setting_id"],
        tup["complexity"],
        tup["fundamentals"],
        tup["role_madness"] # ??
    )

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    break

print(f"Num errors: {cc}")

KeyError: 'game_format'

In [34]:
cc = 0 

for row in playergame.iterrows():
    tup = row[1]
    ins = "INSERT INTO playergame (pg_id, player_id, game_id, alignment_id, death_id, first_hit, last_hit, num_hits, win, pinchhitter, inactive) VALUES('{}','{}','{}','{}', '{}', '{}', '{}', '{}', '{}', '{}')" #, '{}')"

    ins = ins.format(
        tup["pg_id"], 
        tup["player_id"], 
        tup["game_id"], 
        tup["alignment_id"], 
        tup["death_id"],
        tup["first_hit"], 
        tup["last_hit"], 
        tup["num_hits"],
        tup["win"], 
        #tup["pinchhitter"], # not yet
        tup["inactive"]
    )

    if db == 1:
        try: 
            cursor.execute(ins)
        except:
            print(tup)
            print(ins)
            cc = cc + 1

    else: 
        print(ins)
    
    break

print(f"Num errors: {cc}")

INSERT INTO playergame (pg_id, player_id, game_id, alignment_id, death_id, first_hit, last_hit, num_hits, win, pinchhitter, inactive) VALUES('1','1','0.0','0.0', '1.0', '1.0', '1.0', '1.0', 'L', 'nan')
Num errors: 0
